# Books Database Creation by Wesley Giles

![ERD for books database](ERB.png)

## Import necessary modules and environment variables

In [3]:
import pandas as pd
import numpy as np
import sqlalchemy
import sqlalchemy_utils
import os
from dotenv import load_dotenv
load_dotenv()

True

## Create the engine for our database

In [4]:
database_uri = f"{os.environ['DATABASE_BASE_URI']}books"
engine = sqlalchemy.create_engine(database_uri)

## Now let's make a database!

### Start with actually creating the database

In [7]:
try:
  sqlalchemy_utils.create_database(database_uri)
except:
  pass
sqlalchemy_utils.database_exists(database_uri)

True

### Now to get the data to make some tables

In [13]:
users = pd.read_csv("https://docs.google.com/spreadsheets/d/1_c2WTx_eiH8pUM-PTgyt7T4aIl1A3Cp1ukPVPEijoYc/gviz/tq?tqx=out:csv&sheet=users")
books = pd.read_csv("https://docs.google.com/spreadsheets/d/1_D-vW7GXiQfG6D9nzjscgVctKLb6TZl_o8ERNH_tet8/gviz/tq?tqx=out:csv&sheet=books")
authors = pd.read_csv("https://docs.google.com/spreadsheets/d/17rABPt5eaIxfhGO75dYCbH-5IloKsAR0HH9V6VC43ZI/gviz/tq?tqx=out:csv&sheet=authors")
favorites = pd.read_csv("https://docs.google.com/spreadsheets/d/1SLb3RAhcrZsPWRwR0_njWX7KssUYZ16JFsVqBkSU2GI/gviz/tq?tqx=out:csv&sheet=favorite")

table_list = [("users",users),("books",books),("authors",authors),("favorites",favorites)]
for name,table in table_list:
  print(name,table.columns)

users Index(['id', 'first_name', 'last_name', 'email'], dtype='object')
books Index(['id', 'title', 'author_id'], dtype='object')
authors Index(['id', 'author_name'], dtype='object')
favorites Index(['user_id', 'book_id'], dtype='object')


### Now let's make them into tables

In [14]:
for name,table in table_list:
  table.to_sql(name, engine, if_exists = "replace")

pd.read_sql_query("SHOW TABLES;", engine)

,Tables_in_books
0,authors
1,books
2,favorites
3,users


### Now for a final test!

In [17]:
query = """
SELECT books.title, favorites.user_id
FROM books
  JOIN favorites
    ON books.id = favorites.book_id
WHERE favorites.user_id =
    (SELECT users.id FROM users WHERE (users.last_name = "Doe" AND users.first_name = "John"));
"""
pd.read_sql_query(query, engine)

,title,user_id
0,The Shining,1
1,It,1
2,The Great Gatsby,1
